# AWS Academy Machine Learning Foundations - Amazon Forecast Lab

## STOP! - This file is automatically run on lab startup. Do not attempt to run the cells!

This Jupyter notebook is part of the Amazon Forecast student lab. It was ran when the lab was created.

If you just started the lab, load `forecast-lab.ipynb` and work through that notebook instead of this one.

If you were directed to this file, read through the cells and explanations. Avoid running the cells.


## Notebook summary

This notebook loads and preprocesses the online retail dataset. The data is uploaded to Amazon Simple Storage Service (Amazon S3), where it is used to create a forecast through Amazon Forecast. The notebook performs the following steps:

- **Importing and functions** imports the packages used and creates helper functions.
- **Importing data** downloads and loads the data into a pandas DataFrame.
- **Data preprocessing** filters the data that is ready for training
- **Generating training and testing DataFrames** downsamples the data to a daily frequency and splits the dataset into training and testing DataFrames.
- **Uploading to Amazon S3** uploads the DataFrames to Amazon S3 as comma-separated values (CSV) files.
- **Creating the Amazon Forecast dataset group** creates the project dataset group.
- **Creating the datasets** creates the datasets in the dataset group and waits for the import to complete.
- **Creating the predictor** trains the predictor by using the dataset group.
- **Getting accuracy metrics** displays the metrics for the predictor.
- **Creating the forecast** creates a test forecast.
- **Optional cleanup** can perform a cleanup if it isn't completed in the `forecast-lab.ipynb` notebook.

This notebook takes between 60–90 minutes to complete.


## Importing and functions

The following code imports these packages:

- *boto3* represents the AWS SDK for Python (Boto3), which is the Python library for AWS
- *pandas* provides DataFrames for manipulating time series data
- *matplotlib* provides plotting functions
- *sagemaker* represents the API that's needed to work with Amazon SageMaker
- *time*, *sys*, *os*, *io*, and *json* provide helper functions 

In addition, two helper functions are created:

- `upload_s3_csv` uploads pandas DataFrames to Amazon S3 as CSV files. The header is removed, but *not* the index.
- `StatusIndicator` provides a status function for long-running API calls to Amazon Forecast.



In [ ]:
import warnings
warnings.filterwarnings('ignore')
bucket_name='c95096a2133910l4766086t1w419842340-forecastbucket-piqwf7cgzyor'

import boto3
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import time, sys, os, io, json
import sagemaker
!pip3 install pandas==1.5.3

%store bucket_name

s3_resource = boto3.Session().resource('s3')

def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(filename, header=False, index=True)
    dataframe.to_csv(csv_buffer, header=False, index=True )
    s3_resource.Bucket(bucket_name).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

class StatusIndicator:
    
    def __init__(self):
        self.previous_status = None
        self.need_newline = False
        
    def update( self, status ):
        if self.previous_status != status:
            if self.need_newline:
                sys.stdout.write("\n")
            sys.stdout.write( status + " ")
            self.need_newline = True
            self.previous_status = status
        else:
            sys.stdout.write(".")
            self.need_newline = True
        sys.stdout.flush()

    def end(self):
        if self.need_newline:
            sys.stdout.write("\n")

## Importing data

The following cell downloads the dataset, which is an Microsoft Excel file. This file is loaded into pandas as a DataFrame.

In [ ]:

session = boto3.Session()
forecast = session.client(service_name='forecast') 
forecast_query = session.client(service_name='forecastquery')


## Data preprocessing

The following cell completes the following preprocessing steps:

- Removes instances with missing values
- Sets the index to the InvoiceDate feature
- Removes instances that aren't from the United Kingdom
- Removes instances that don't use the target stock code (21232)
- Keeps instances where the price is greater than 0



In [ ]:
retail = pd.read_excel('online_retail_II.xlsx',engine='openpyxl')
retail = retail.dropna()
retail['InvoiceDate'] = pd.to_datetime(retail.InvoiceDate)
retail = retail.set_index('InvoiceDate')

country_filter = ['United Kingdom']
retail = retail[retail['Country'].isin(country_filter)]

#stockcodes = ['ADJUST', 'ADJUST2', 'POST', 'M']
#stockcodes = [21232,22423]
stockcodes = [21232]
retail = retail[retail.StockCode.isin(stockcodes)]

retail = retail[retail['Price']>0]

## Generating the training and testing DataFrames

The following cell:

- Splits the data into time series and related times series pandas DataFrames.
- Downsamples the data from multiple sales entries per day into a single daily value. The **Quantity** column is summed, and the mean is used for the **Price** column.
- Splits the DataFrames into a training set of data from January 2010–October 2010, and a testing set of data from November 2010–December 2010.



In [ ]:

retail_timeseries = retail[['StockCode','Quantity']]

retail_timeseries = retail_timeseries.groupby('StockCode').resample('D').sum().reset_index().set_index(['InvoiceDate'])

df_related_time_series = retail[['StockCode','Price']]
df_related_time_series2 = df_related_time_series.groupby('StockCode').resample('D').mean().reset_index().set_index(['InvoiceDate'])
df_related_time_series3 = df_related_time_series2.groupby('StockCode').pad()

#df_related_time_series4 = df_related_time_series3.reset_index().set_index('InvoiceDate')

# Select January to November for one DataFrame.
jan_to_oct = retail_timeseries['2009-12':'2010-10']
nov_to_dec = retail_timeseries['2010-11':'2010-12']
jan_to_oct_related = df_related_time_series2['2009-12':'2010-10']

In [ ]:
df_related_time_series2.head()

## Uploading to Amazon S3

The following cell uploads the DataFrames to Amazon S3 by using the helper function that was created earlier.

In [ ]:

prefix='lab_4'
train='../autorun/retail_ts_train.csv'
train_related='../autorun/related_ts_train.csv'
test='../autorun/retail_ts_test.csv'

key=prefix + '/forecast/' + train
# key='lab_4_forecast_t/forecast/retail_time_series_train.csv'
related_key = prefix + '/forecast/' + train_related
# related_key='lab_4_forecast_t/forecast/related.csv'

upload_s3_csv(train, 'forecast', jan_to_oct)
upload_s3_csv(train_related, 'forecast', jan_to_oct_related)
upload_s3_csv(test, 'forecast', nov_to_dec)

dataset_frequency = "D" 
timestamp_format = "yyyy-MM-dd"

# project = prefix
dataset_name = prefix+'_ds'
related_dataset_name = prefix+'_rds'
dataset_group_name = prefix +'_dsg'

s3_data_path = "s3://"+bucket_name+"/"+key
s3_related_data_path = "s3://"+bucket_name+"/"+related_key

In [ ]:
jan_to_oct_related.head()

In [ ]:

%store prefix
%store train
%store test
%store key

## Creating the Amazon Forecast dataset group

The following cell creates the dataset group for the forecast.

In [ ]:
dataset_group_arn = None
dsgs = forecast.list_dataset_groups()
for dsg in dsgs['DatasetGroups']:
    if dsg['DatasetGroupName']==dataset_group_name:
        dataset_group_arn=dsg['DatasetGroupArn']

In [ ]:
if dataset_group_arn is None:
    create_dataset_group_response = forecast.create_dataset_group(DatasetGroupName=dataset_group_name, Domain="RETAIL" )
    dataset_group_arn = create_dataset_group_response['DatasetGroupArn']

## Creating the datasets

The following cell creates the time series and related datasets, and adds them to the dataset group.

The cell will wait loop and display the status until the datasets are created.

In [ ]:
iam = boto3.resource('iam')
role_arn = iam.Role('ForecastRole').arn

# This is the schema of the time series dataset.
schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      },
      {
         "AttributeName":"demand",
         "AttributeType":"float"
      }
   ]
}

In [ ]:
dataset_arn = None
dataset_list = forecast.list_datasets()
for dataset in dataset_list['Datasets']:
    if dataset['DatasetName']==dataset_name:
        dataset_arn = dataset['DatasetArn']

In [ ]:
if dataset_arn is None:
    time_series_response=forecast.create_dataset(
                        Domain="RETAIL",
                        DatasetType='TARGET_TIME_SERIES',
                        DatasetName=dataset_name,
                        DataFrequency=dataset_frequency, 
                        Schema = schema
    )

    dataset_arn = time_series_response['DatasetArn']

In [ ]:
# Create the import job for the time series dataset.
dataset_import_job_name = 'EP_DSIMPORT_JOB_TARGET'
ds_import_job_arn = None
dataset_import_job_list = forecast.list_dataset_import_jobs()

for dataset_import_job in dataset_import_job_list['DatasetImportJobs']:
    if dataset_import_job['DatasetImportJobName'] == dataset_import_job_name:
        ds_import_job_arn = dataset_import_job['DatasetImportJobArn']

In [ ]:
# If the import job doesn't already exist, create it
if ds_import_job_arn is None:
    data_source = {"S3Config" : {"Path":s3_data_path,"RoleArn": role_arn} }
    ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=dataset_import_job_name,
                                                          DatasetArn=dataset_arn,
                                                          DataSource= data_source,
                                                          TimestampFormat=timestamp_format
                                                         )
    ds_import_job_arn=ds_import_job_response['DatasetImportJobArn']

In [ ]:
print(ds_import_job_arn)

In [ ]:
    
# This is the schema of the related data, which contains the price.
related_schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      },
      {
         "AttributeName":"price",
         "AttributeType":"float"
      }
   ]
}



In [ ]:
related_dataset_arn = None
dataset_list = forecast.list_datasets()
for dataset in dataset_list['Datasets']:
   if dataset['DatasetName']==related_dataset_name:
        related_dataset_arn = dataset['DatasetArn']

In [ ]:
if related_dataset_arn is None:
    related_time_series_response=forecast.create_dataset(
                        Domain="RETAIL",
                        DatasetType='RELATED_TIME_SERIES',
                        DatasetName=related_dataset_name,
                        DataFrequency=dataset_frequency, 
                        Schema = related_schema)
    related_dataset_arn = related_time_series_response['DatasetArn']

In [ ]:
# forecast.describe_dataset(DatasetArn=related_dataset_arn)


related_dataset_import_job_name = 'EP_DSIMPORT_JOB_TARGET_RELATED'
ds_related_import_job_arn = None
related_data_source = {"S3Config" : {"Path":s3_related_data_path,"RoleArn": role_arn} }

dataset_import_job_list = forecast.list_dataset_import_jobs()

for dataset_import_job in dataset_import_job_list['DatasetImportJobs']:
    print(dataset_import_job)
    if dataset_import_job['DatasetImportJobName']==related_dataset_import_job_name:
        if dataset_import_job['Status'] == 'ACTIVE':
            ds_related_import_job_arn = dataset_import_job['DatasetImportJobArn']
        elif dataset_import_job['Status'] in ['CREATE_FAILED']:
            forecast.delete_dataset_import_job(DatasetImportJobArn=dataset_import_job['DatasetImportJobArn'])
            time.sleep(60)
        else:
            pass
            

In [ ]:
print(ds_related_import_job_arn)

In [ ]:

if ds_related_import_job_arn is None:
    ds_related_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=related_dataset_import_job_name,
                                                          DatasetArn=related_dataset_arn,
                                                          DataSource= related_data_source,
                                                          TimestampFormat=timestamp_format
                                                         )
    ds_related_import_job_arn=ds_related_import_job_response['DatasetImportJobArn']

In [ ]:
# Add the time series and related dataset to the dataset group.
dsg = forecast.describe_dataset_group(DatasetGroupArn=dataset_group_arn)
print(dsg)

In [ ]:
print(dataset_arn)
print(related_dataset_arn)

In [ ]:
forecast.update_dataset_group(DatasetGroupArn=dataset_group_arn, DatasetArns=[dataset_arn, related_dataset_arn])

In [ ]:
# Wait for the related dataset to finish.
status_indicator = StatusIndicator()

while True:
    status = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_related_import_job_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

In [ ]:
# Wait for the time series dataset to finish. This process typically takes longer than the related set.
status_indicator = StatusIndicator()

while True:
    status = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

The following cell stores the Amazon Resource Names (ARNs) for the forecast objects that were created previously. They can be loaded from other notebooks.

In [ ]:
%store ds_import_job_arn
%store dataset_arn
%store dataset_group_arn
%store related_dataset_arn
%store ds_related_import_job_arn

## Creating the predictor

The following cell creates the predictor by using the following parameters:

- The forecast horizon is set to *30 days*.
- *DeepAR+* is the selected algorithm. For more information, see [DeepAR+ Algorithm](https://docs.aws.amazon.com/forecast/latest/dg/aws-forecast-recipe-deeparplus.html) in the AWS Documentation.
- Hyperparameters are specified for the algorithm. These hyperparameters were generated by running the forecast with PerformHPO set to *true*. This created a hyperparameter tuning job on the model, which produced the values that follow.
- A single backtest window for *30 days* is used.
- The input data configuration is set to the dataset group that was created earlier.
- Holidays for the United Kingdom are added as supplementary features.
- A featurization pipeline is created for the price features. For more information, see the [Handling Missing Values](https://docs.aws.amazon.com/forecast/latest/dg/howitworks-missing-values.html) topic in the AWS Documentation.

The cell will wait loop and display the status until the datasets are created.

In [ ]:
predictor_name= prefix+'_deeparp_algo'
forecast_horizon = 30
algorithm_arn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

training_parameters =  {'context_length': '172', 
                        'epochs': '500', 
                        'learning_rate': '0.00023391131837525837', 
                        'learning_rate_decay': '0.5', 
                        'likelihood': 'student-t', 
                        'max_learning_rate_decays': '0', 
                        'num_averaged_models': '1', 
                        'num_cells': '40', 
                        'num_layers': '2', 
                        'prediction_length': '30'}

evaluation_parameters= {"NumberOfBacktestWindows": 1, "BackTestWindowOffset": 30}

input_data_config = {"DatasetGroupArn": dataset_group_arn, "SupplementaryFeatures": [ {"Name": "holiday","Value": "UK"} ]}
                  
featurization_config= {"ForecastFrequency": dataset_frequency,
                      "Featurizations": 
                      [
                          {
                            "AttributeName": "price",
                            "FeaturizationPipeline": [
                                {
                                    "FeaturizationMethodName": "filling",
                                    "FeaturizationMethodParameters": {
                                        "middlefill": "median",
                                        "backfill": "min",
                                        "futurefill": "max"               
                                        }
                                }
                            ]
                        }
                      ]}

predictor_arn = None
predictors = forecast.list_predictors()
for predictor in predictors['Predictors']:
    print(predictor)
    if predictor['PredictorName'] == predictor_name:
        predictor_arn = predictor['PredictorArn']

if predictor_arn is None:
    create_predictor_response=forecast.create_predictor(PredictorName = predictor_name, 
                                                  AlgorithmArn = algorithm_arn,
                                                  ForecastHorizon = forecast_horizon,
                                                  PerformAutoML = False,
                                                  PerformHPO = False,
                                                  EvaluationParameters= evaluation_parameters, 
                                                  InputDataConfig = input_data_config,
                                                  FeaturizationConfig = featurization_config,
                                                  TrainingParameters = training_parameters
                                                 )
    predictor_arn = create_predictor_response['PredictorArn']

%store predictor_arn

In [ ]:
status_indicator = StatusIndicator()
while True:
    status = forecast.describe_predictor(PredictorArn=predictor_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

In [ ]:
f = forecast.describe_predictor(PredictorArn=predictor_arn)
print(f['TrainingParameters'])

## Getting accuracy metrics

The next cell prints the accuracy metrics for the predictor that was just created.

In [ ]:
forecast.get_accuracy_metrics(PredictorArn=predictor_arn)

## Creating the forecast

The following cell creates a forecast from the predictor that was created earlier. 

The predictor and forecast ARN values are stored so that they can be retrieved from the lab notebook.



In [ ]:
forecast_Name= prefix+'_deeparp_algo_forecast'
forecast_arn = None
forecasts = forecast.list_forecasts()
for f in forecasts['Forecasts']:
    if f['ForecastName']==forecast_Name:
        forecast_arn = f['ForecastArn']
        
if forecast_arn is None:
    create_forecast_response=forecast.create_forecast(ForecastName=forecast_Name,
                                                  PredictorArn=predictor_arn)
    forecast_arn = create_forecast_response['ForecastArn']

In [ ]:
%store forecast_arn


In [ ]:
status_indicator = StatusIndicator()
while True:
    status = forecast.describe_forecast(ForecastArn=forecast_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

print(forecast_arn)

The next cell creates a quick forecast as a test, which can be useful for troubleshooting.

In [ ]:
print()
forecast_response = forecast_query.query_forecast(
    ForecastArn=forecast_arn,
    Filters={"item_id":"21232"}
)
print(forecast_response)

## Optional cleanup

Cleanup is performed in the `forecast-lab.ipynb` notebook. If you must perform the cleanup here, change the following cell to code by selecting the cell and pressing Y. Then, run the cell.

forecast.delete_forecast(ForecastArn=forecast_arn)
time.sleep(60)

forecast.delete_predictor(PredictorArn=predictor_arn)
time.sleep(60)

forecast.delete_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)
time.sleep(60)

forecast.delete_dataset(DatasetArn=dataset_arn)
time.sleep(60)

forecast.delete_dataset_group(DatasetGroupArn=dataset_group_arn)